<a href="https://colab.research.google.com/github/yashkapur0403/Neural-Networks-Practise/blob/main/CNN_model_lung_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
import zipfile, os, cv2
import numpy as np

drive.mount('/content/drive')

# allow permission
zip_path = "/content/drive/My Drive/Colab Notebooks/chest_xray.zip"
extract_path = "/content/chest_xray"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# STEP 2: Load images and prepare data
def load_images_from_folder(folder_path, label):
    data = []
    for filename in os.listdir(folder_path):
        path = os.path.join(folder_path, filename)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            img_flat = img.flatten() / 255.0
            data.append((img_flat, label))
    return data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Data loading
transform = transforms.Compose([
    transforms.Grayscale(),                # Convert to 1-channel
    transforms.Resize((128, 128)),         # Resize images
    transforms.ToTensor(),                 # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))   # Normalize to [-1, 1]
])

# Fix: Point to the train directory, not individual class folders
train_dataset = datasets.ImageFolder("/content/chest_xray/chest_xray/train", transform=transform)
test_dataset = datasets.ImageFolder("/content/chest_xray/chest_xray/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # [B, 16, 128, 128]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B, 16, 64, 64]
            nn.Conv2d(16, 32, kernel_size=3, padding=1), # [B, 32, 64, 64]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B, 32, 32, 32]
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*32*32, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x

# Final training of model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):  # You can increase this
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

#Accuracy
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = (outputs >= 0.5).squeeze(1).int()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total * 100:.2f}%")

Epoch 1, Loss: 0.3340
Epoch 2, Loss: 0.2275
Epoch 3, Loss: 0.1982
Epoch 4, Loss: 0.1733
Epoch 5, Loss: 0.1514
Epoch 6, Loss: 0.1417
Epoch 7, Loss: 0.1257
Epoch 8, Loss: 0.1114
Epoch 9, Loss: 0.0992
Epoch 10, Loss: 0.0880
Epoch 11, Loss: 0.0781
Epoch 12, Loss: 0.0735
Epoch 13, Loss: 0.0681
Epoch 14, Loss: 0.0633
Epoch 15, Loss: 0.0555
Epoch 16, Loss: 0.0469
Epoch 17, Loss: 0.0431
Epoch 18, Loss: 0.0386
Epoch 19, Loss: 0.0357
Epoch 20, Loss: 0.0332
Test Accuracy: 73.72%
